<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 4.3: Common Pass Idioms

**Prev: [FIRRTL AST Traversal](4.2_firrtl_ast_traversal.ipynb)**<br>
**Next: [A FIRRTL Transform Example](4.4_firrtl_add_ops_per_module.ipynb)**

### Adding statements
假设我们想编写一个 pass 来拆分嵌套的 DoPrim 表达式，从而将这个：
```
circuit Top:
  module Top :
  input x: UInt<3>
  input y: UInt<3>
  input z: UInt<3>
  output o: UInt<3>
  o <= add(x, add(y, z))
```
转换为这个：
```
circuit Top:
  module Top :
  input x: UInt<3>
  input y: UInt<3>
  input z: UInt<3>
  output o: UInt<3>
  node GEN_1 = add(y, z)
  o <= add(x, GEN_1)
```

我们首先需要遍历 AST 到每个 Statement 和 Expression。然后，当我们看到一个 DoPrim 时，我们需要在模块的主体中添加一个新的 DefNode，并在 DoPrim 的位置插入对该 DefNode 的引用。下面的代码实现了这一点（并保留了 Info 标记）。请注意，`Namespace` 是一个位于 [Namespace.scala](https://github.com/ucb-bar/firrtl/blob/master/src/main/scala/firrtl/Namespace.scala) 的实用函数。

```scala
object Splitter extends Pass {
  def name = "Splitter!"
  /** Run splitM on every module **/
  def run(c: Circuit): Circuit = c.copy(modules = c.modules map(splitM(_)))

  /** Run splitS on the body of every module **/
  def splitM(m: DefModule): DefModule = m map splitS(Namespace(m))

  /** Run splitE on all children Expressions.
    * If stmts contain extra statements, return a Block containing them and 
    *    the new statement; otherwise, return the new statement. */
  def splitS(namespace: Namespace)(s: Statement): Statement = {
    val block = mutable.ArrayBuffer[Statement]()
    s match {
      case s: HasInfo => 
        val newStmt = s map splitE(block, namespace, s.info)
        block.length match {
          case 0 => newStmt
          case _ => Block(block.toSeq :+ newStmt)
        }
      case s => s map splitS(namespace)
  }

  /** Run splitE on all children expressions.
    * If e is a DoPrim, add a new DefNode to block and return reference to
    * the DefNode; otherwise return e.*/
  def splitE(block: mutable.ArrayBuffer[Statement], namespace: Namespace, 
             info: Info)(e: Expression): Expression = e map splitE(block, namespace, info) match {
    case e: DoPrim =>
      val newName = namespace.newTemp
      block += DefNode(info, newName, e)
      Ref(newName, e.tpe)
    case _ => e
  }
}
```
### Deleting statements
假设我们想编写一个 pass，将所有值为文字的 DefNode 内联，从而将这个：
```
circuit Top:
  module Top :
  input x: UInt<3>
  output o: UInt<4>
  node y = UInt(1)
  o <= add(x, y)
```
转换为这个：
```
circuit Top:
  module Top :
  input x: UInt<3>
  output y: UInt<4>
  o <= add(x, UInt(1))
```

我们首先需要遍历 AST 到每个 Statement 和 Expression。然后，当我们看到一个指向文字的 DefNode 时，我们需要将其存储到哈希图中并返回一个 EmptyStmt（从而删除该 DefNode）。然后，每当我们看到对已删除的 DefNode 的引用时，我们必须插入相应的文字。

```scala
object Inliner extends Pass {
  def name = "Inliner!"
  /** Run inlineM on every module **/
  def run(c: Circuit): Circuit = c.copy(modules = c.modules map(inlineM(_)))

  /** Run inlineS on the body of every module **/
  def inlineM(m: DefModule): DefModule = m map inlineS(mutable.HashMap[String, Expression]())

  /** Run inlineE on all children Expressions, and then run inlineS on children statements.
    * If statement is a DefNode containing a literal, update values and
    *   return EmptyStmt; otherwise return statement. */
  def inlineS(values: mutable.HashMap[String, Expression])(s: Statement): Statement =
    s map inlineE(values) map inlineS(values) match {
      case d: DefNode => d.value match {
        case l: Literal =>
          values(d.name) = l
          EmptyStmt
        case _ => d
      }
      case o => o 
    }

  /** If e is a reference whose name is contained in values, 
    *   return values(e.name); otherwise run inlineE on all 
    *   children expressions.*/
  def inlineE(values: mutable.HashMap[String, Expression])(e: Expression): Expression = e match {
    case e: Ref if values.contains(e.name) => values(e.name)
    case _ => e map inlineE(values)
  }
}
```

### Add a Primop
这有用吗？通过向 [firrtl repo](https://github.com/freechipsproject/firrtl) 提交问题让 [@azidar](https://github.com/azidar) 知道！

### Swap a statement
这有用吗？通过向 [firrtl repo](https://github.com/freechipsproject/firrtl) 提交问题让 [@azidar](https://github.com/azidar) 知道！